## Импорт библиотек

In [ ]:
import time
# Время старта работы ноутбука
notebook_starttime = time.time()

In [ ]:
import os
import subprocess
import gc
import pickle
from itertools import zip_longest

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import VotingRegressor

from joblib import dump
from joblib import load

import catboost as cb

import optuna

import matplotlib.pyplot as plt

#from mlxtend.evaluate.time_series import GroupTimeSeriesSplit, plot_splits

## Настройка: сабмит или локально

In [ ]:
# Ставим is_local в True, если локально работаем, если сабмитим - ставим в False
is_local = False
#is_local = True

# Ставим is_gpu в True, если будем работать на GPU, если на процессоре - ставим в False
# is_gpu = False
is_gpu = True

# Ставим is_tuning в True, если запускаем подбор гиперпараметров в Optuna
is_tuning = False

# Начальная дата обучения модели
training_start_date = 'datetime >= "2022-01-01 00:00:00"'


# Устанавливаем время начало переобучения. Начинаем переобучать модели заново,
# когда подходит дата предсказаний, которая идет в скор.
# scor_start_time = pd.to_datetime('2023-06-01')

# Устанавливаем время завершения переобучения и предсказаний.
# после этой даты просто возвращаем те предсказания что дали.
# scor_stop_time = pd.to_datetime('2023-09-01')

# Максимальное время работы ноутбука, когда еще можно тренировать модели
max_train_duration = (8*60*60 + 60*50)

# Время после которого не осуществляем тренировку моделей
stop_train_time = notebook_starttime + max_train_duration

# Ставим в False, если не хотитим отключать контроль времени выполнения ноутбука
# (не обучать модели заново после 8 часов 30 минут)
# Если хотим, чтобы модели обучались заново и после лимита, ставим True
#is_disable_run_time_limit = True
is_disable_run_time_limit = False

In [ ]:
# Настройки графического процессора
if is_local:
    # Для домашней машины
    import lightgbm as lgb
    # Число процессов параллельного обучения модетей и предсказания
    #num_processes = 6
    num_processes = 1
    # число параллельных threads для тренировки каждой модели
    n_jobs = None
    #n_jobs = 1
    # Тип ускорителя в системе: gpu или cuda
    gpu_type = 'gpu'
    # число GPU в системе
    gpus_n = 1
else:
    # Для карточки на кагле T4x2
    !pip uninstall -y lightgbm
    !pip install /kaggle/input/lightgbm420-cuda/lightgbm-4.2.0-py3-none-manylinux_2_35_x86_64.whl
    import lightgbm as lgb
    
    num_processes = 2
    # число параллельных threads для тренировки каждой модели
    n_jobs = None
    # Тип ускорителя в системе: gpu или cuda
    gpu_type = 'cuda'
    # число GPU в системе
    gpus_n = 2

In [ ]:
print("Версия lightgbm:", lgb.__version__)

In [ ]:
# Возвращает сколько уже работает ноутбук
def p_time():
    run_time = round(time.time() - notebook_starttime)
    return str(run_time).zfill(5)+' sec:'

In [ ]:
class MonthlyKFold:
    def __init__(self, n_splits=3):
        self.n_splits = n_splits
        
    def split(self, X, y, groups=None):
        dates = 12 * X["year"] + X["month"]
        timesteps = sorted(dates.unique().tolist())
        X = X.reset_index()
        
        for t in timesteps[-self.n_splits:]:
            idx_train = X[dates.values < t].index
            idx_test = X[dates.values == t].index
            
            yield idx_train, idx_test
            
    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

In [ ]:
if is_tuning:
    cv = MonthlyKFold()
    CV = GroupTimeSeriesSplit(test_size=3, n_splits=3, window_type='rolling', shift_size=2)

## Feature Engineering sub

In [ ]:
def feature_eng(df_data, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days):
    print(p_time(), 'feature_eng: Start')
    working_days = (
        working_days
        .with_columns(
            pl.col("date").cast(pl.Date)
        )
    )
    
    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )
    
    df_client = (
        df_client
        .with_columns(
            (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
        )
    )
    
    df_gas = (
        df_gas
        .rename({"forecast_date": "date"})
        .with_columns(
            (pl.col("date") + pl.duration(days=1)).cast(pl.Date)
        )
    )
    
    df_electricity = (
        df_electricity
        .rename({"forecast_date": "datetime"})
        .with_columns(
            pl.col("datetime") + pl.duration(days=1)
        )
    )
    
    df_location = (
        df_location
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )
    
    df_forecast = (
        df_forecast
        .rename({"forecast_datetime": "datetime"})
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            #pl.col('datetime').dt.convert_time_zone("Europe/Bucharest").dt.replace_time_zone(None).cast(pl.Datetime("us")),
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us"))
            #pl.col('datetime').cast(pl.Datetime)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_historical = (
        df_historical
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            pl.col("datetime") + pl.duration(hours=37)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
        #.with_columns(
        #    (pl.col("direct_solar_radiation")+pl.col("diffuse_radiation")).alias("rad_sum"),
        #)
    )
    
    df_forecast_date = (
        df_forecast
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_forecast_local = (
        df_forecast
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    df_historical_date = (
        df_historical
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_historical_local = (
        df_historical
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    # Объединение всех обработанных данных с основным датафреймом df_data
    df_data = (
        df_data
        .join(df_gas, on="date", how="left")
        .join(df_client, on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity, on="datetime", how="left")
        
        .join(df_forecast_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_historical_date, on="datetime", how="left", suffix="_hd")
        .join(df_historical_local, on=["county", "datetime"], how="left", suffix="_hl")
        

        .join(df_forecast_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fd7")
        .join(df_forecast_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_fl7")
        .join(df_historical_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hd7")
        .join(df_historical_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hl7")

        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        #.join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=8)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=9)).rename({"target": "target_8"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=10)).rename({"target": "target_9"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=11)).rename({"target": "target_10"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=12)).rename({"target": "target_11"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=13)).rename({"target": "target_12"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_13"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=353)).rename({"target": "target_y_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=361)).rename({"target": "target_y_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=362)).rename({"target": "target_y_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=363)).rename({"target": "target_y_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=364)).rename({"target": "target_y_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=365)).rename({"target": "target_y_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=366)).rename({"target": "target_y_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=367)).rename({"target": "target_y_8"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        
        #Добавляем коолонку рабочий день или нет.
        .join(working_days, on="date", how="left")
        # Создание категориальных признаков и тригонометрических функций времени
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"), # Добавление номера дня в году
            pl.col("datetime").dt.hour().alias("hour"),# Добавление часа
            pl.col("datetime").dt.day().alias("day"),# Добавление дня
            pl.col("datetime").dt.weekday().alias("weekday"),# Добавление дня недели
            pl.col("datetime").dt.month().alias("month"),# Добавление месяца
            pl.col("datetime").dt.year().alias("year"),# Добавление года
        )
        # Приведение типов данных
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("category_1"),
        )
        
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"), # Тригонометрические функции для дня в году
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )

        # Новая фича. Назвал cap_rad_temp. нашел в обсуждениях: 
        # https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/468654
        .with_columns(
            (pl.col("installed_capacity")*pl.col("surface_solar_radiation_downwards")/(pl.col('temperature') + 273.15)).alias("cap_rad_temp"),
        )

    )

    for hours_lag in [1, 2]:
            df_data = df_data.join(
                df_forecast_date.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on="datetime",
                how="left",
                suffix=f"_forecast_{hours_lag}h",
            )
            df_data = df_data.join(
                df_forecast_local.with_columns(
                    pl.col("datetime") + pl.duration(hours=hours_lag)
                ),
                on=["county", "datetime"],
                how="left",
                suffix=f"_forecast_local_{hours_lag}h",
            )
    df_data = (
        # Удаление ненужных колонок
        df_data.drop("date", "datetime", "hour", "dayofyear")
    )
    '''
    # Делаем колонку со сдвигом на один день для cap_rad_temp
    df_cap_rad_temp = df_data[['cap_rad_temp',"county", "is_business", "product_type", "is_consumption", "datetime"]]
    df_data = (
        df_data
        .join(df_cap_rad_temp.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"cap_rad_temp": "cap_rad_temp_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        # Удаление ненужных колонок
        .drop("date", "datetime", "hour", "dayofyear")
    )
    '''
    print(p_time(), 'feature_eng: End')
    # return df_data, df_historical_local
    return df_data

In [ ]:
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "category_1"]
    print(p_time(), 'to_pandas: Start')
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    

    print(p_time(), 'to_pandas: Start make Features')

    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    

    #df["target_skew_1"] = df[[f"target_{i}" for i in range(1, 14)]].skew(1)

    df["target_ratio_1"] = df["target_1"] / (df["target_13"] + 1e-3)
    df["target_mean_1"] = df[[f"target_{i}" for i in range(1, 14)]].mean(1)
    df["target_std_1"] = df[[f"target_{i}" for i in range(1, 14)]].std(1)

    df["target_ratio_2"] = df["target_1"] / (df["target_7"] + 1e-3)
    df["target_mean_2"] = df[[f"target_{i}" for i in range(1, 8)]].mean(1)
    df["target_std_2"] = df[[f"target_{i}" for i in range(1, 8)]].std(1)

    df["target_ratio_y_1"] = df["target_y_1"] / (df["target_y_8"] + 1e-3)
    df["target_mean_y_1"] = df[[f"target_y_{i}" for i in range(1, 9)]].mean(1)
    df["target_std_y_1"] = df[[f"target_y_{i}" for i in range(1, 9)]].std(1)
    #df["target_ratio_y_2"] = df["target_y_2"] / (df["target_y_3"] + 1e-3)
    print(p_time(), 'to_pandas: End')
    
    return df

In [ ]:
target_list = ['target_{}'.format(i) for i in range(1, 13+1)]

print(target_list)

In [ ]:
root = "/kaggle/input/predict-energy-behavior-of-prosumers"

# Для локальных вычислений. Последний data_block_id тренировочной выборки
# А начиная со следующего data_block_id и до конца идет тест
# train_end_data_block_id = 517
# train_end_data_block_id = 600+360
# train_end_data_block_id = 600
train_end_data_block_id = 598
#train_end_data_block_id = 636
# train_end_data_block_id = 680

data_cols        = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id', 'data_block_id']
# В df_data_cols колонки в таком порядке в каком они потом формируются в df_data
df_data_cols     = ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'datetime', 'data_block_id', 'row_id']
client_cols      = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_cols         = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_cols = ['forecast_date', 'euros_per_mwh']
forecast_cols    = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_cols  = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols    = ['longitude', 'latitude', 'county']
target_cols      = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

save_path = None
load_path = None

In [ ]:
# для оптуны
# Диапазон гиперпараметров для модели обучающейся на данных is_consumption=1 
def lgb_objective_cons1(trial):
    params = {
        'device'            : trial.suggest_categorical('device', ['gpu']),
        'gpu_platform_id'   : trial.suggest_int('gpu_platform_id', 1, 1),
        'gpu_device_id'     : trial.suggest_int('gpu_device_id', 0, 0),
        'n_estimators'      : trial.suggest_int('n_estimators', 1000, 2000),
        'verbose'           : trial.suggest_int('verbose', -1, -1),
        'random_state'      : trial.suggest_int('random_state', 42, 42),
        'objective'         : trial.suggest_categorical('objective', ['l2']),
        'num_leaves'        : trial.suggest_int('num_leaves', 20, 50),
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        #'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 20.0),
        'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 10.0),
        'reg_lambda'        : trial.suggest_float('reg_lambda', 1e-2, 20.0),
        'min_child_samples' : trial.suggest_int('min_child_samples', 4, 256),
        'max_depth'         : trial.suggest_int('max_depth', -1, -1),
        'max_bin'           : trial.suggest_int('max_bin', 32, 200),
    }
    
    model  = lgb.LGBMRegressor(**params)
    X      = df_train[df_train['is_consumption']==1].drop(columns=["target", "datetime"]).reset_index(drop=True)
    y      = df_train[df_train['is_consumption']==1]["target"].reset_index(drop=True)
    scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
    
    return -np.mean(scores)

# Диапазон гиперпараметров для модели обучающейся на данных is_consumption=0
def lgb_objective_cons0(trial):
    params = {
        'device'            : trial.suggest_categorical('device', ['gpu']),
        'gpu_platform_id'   : trial.suggest_int('gpu_platform_id', 1, 1),
        'gpu_device_id'     : trial.suggest_int('gpu_device_id', 0, 0),
        'n_estimators'      : trial.suggest_int('n_estimators', 1000, 2000),
        'verbose'           : trial.suggest_int('verbose', -1, -1),
        'random_state'      : trial.suggest_int('random_state', 42, 42),
        'objective'         : trial.suggest_categorical('objective', ['l2']),
        'num_leaves'        : trial.suggest_int('num_leaves', 20, 50),
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        #'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 20.0),
        #'reg_lambda'        : trial.suggest_float('reg_lambda', 1e-2, 20.0),
        'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 10.0),
        'reg_lambda'        : trial.suggest_float('reg_lambda', 12, 20.0),
        'min_child_samples' : trial.suggest_int('min_child_samples', 4, 256),
        'max_depth'         : trial.suggest_int('max_depth', -1, -1),
        'max_bin'           : trial.suggest_int('max_bin', 32, 200),
    }
    
    model  = lgb.LGBMRegressor(**params)
    X      = df_train[df_train['is_consumption']==0].drop(columns=["target", "datetime"]).reset_index(drop=True)
    y      = df_train[df_train['is_consumption']==0]["target"].reset_index(drop=True)
    scores = cross_val_score(model, X, y, groups=groups, cv=cv, scoring='neg_mean_absolute_error')
    
    return -np.mean(scores)

## Global Variables

## Исследование

In [ ]:
if is_local:
    # Загрузка данных об энергопотреблении
    train = pd.read_csv(os.path.join(root, "train.csv"))
    
    # Создание сводной таблицы с средними значениями целевой переменной (target)
    # для каждой комбинации даты, округа, типа продукта, бизнеса и потребления
    pivot_train = train.pivot_table(
        index='datetime',
        columns=['county', 'product_type', 'is_business', 'is_consumption'],
        values='target',
        aggfunc='mean'
    )
    
    # Переименование колонок для удобства доступа и интерпретации
    pivot_train.columns = ['county{}_productType{}_isBusiness{}_isConsumption{}'.format(*col) for col in pivot_train.columns.values]
    pivot_train.index = pd.to_datetime(pivot_train.index)
    
    pivot_train

### 2023 год 

In [ ]:
if is_local:
    # Копирование сводной таблицы для визуализации
    df_plot = pivot_train.copy()
    
    # Нормализация данных для визуализации
    df_plot = (df_plot - df_plot.min()) / (df_plot.max() - df_plot.min())
    
    # Ресемплирование данных по дням и вычисление средних значений
    df_plot_resampled_D = df_plot.resample('D').mean()
    
    # Визуализация нормализованных данных с прозрачностью (alpha=0.1)
    df_plot_resampled_D.loc['2022-7':].plot(alpha=0.1, color='green', figsize=(18, 6), legend=False)


In [ ]:
if is_local:
    # Выбор колонок, соответствующих различным категориям потребления
    columns_consumption_0 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption0')]
    columns_consumption_1 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption1')]
    
    # Создание фигуры для визуализации
    plt.figure(figsize=(15, 6))
    
    # Создание пустых линий для легенды
    plt.plot([], color='red', label='is_Consumption = 1')  # Изменено на желтый цвет
    plt.plot([], color='black', label='is_Consumption = 0')   # Изменено на черный цвет
    
    # Отображение легенды
    plt.legend()
    
    # Визуализация данных для 'is_Consumption = 0' черным цветом
    for column in columns_consumption_0:
        df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='black', legend=False)  # Изменено на черный
    
    # Визуализация данных для 'is_Consumption = 1' желтым цветом
    for column in columns_consumption_1:
        df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='red', legend=False)  # Изменено на желтый
    
    # Отображение графика
    plt.show()




## Подготовка данных

### Запись тестовых и тренировочных csv файлов

In [ ]:
if is_local:
    # Если выполняем локально
    # Создаем пути для локальных данных с трейном и тестовой выборкой
    train_path = 'train'
    os.makedirs(train_path, exist_ok=True)
    test_path = 'example_test_files'
    os.makedirs(test_path, exist_ok=True)
    
    # Создаем пути для увеличенных данных по май 2024 года
    full_train_path = 'full_train'
    os.makedirs(full_train_path, exist_ok=True)
    full_test_path = 'full_example_test_files'
    os.makedirs(full_test_path, exist_ok=True)
    full_root_path = 'full_predict-energy-behavior-of-prosumers'
    os.makedirs(full_root_path, exist_ok=True)
else:
    # Если сабмит
    train_path = root
# Путь, куда запишем csv файлы для теста

### Увеличиваем исходные данные по май 2024 года

In [ ]:
# Увеличивает датафрейм по май 2024
# df - датафрейм который увеличиваем
# date_col колонка по которой вырезаем старые данные
# колонки с данными к которым дабавляем 365 дней
def enlarge_df(df_name, date_col, add_date_cols):

    print('Увеличиваем размер для:', df_name)
    df = pd.read_csv(os.path.join(root, df_name))
    for col in add_date_cols:
        df[col] = pd.to_datetime(df[col])
    
    start_date = df[date_col].max() + pd.DateOffset(days=1)
    # Конечная дата по которую увеличить датафрейм    
    end_date = pd.to_datetime('2024-05-01')
    
    old_start_date = start_date - pd.DateOffset(days=365)
    old_end_date = end_date - pd.DateOffset(days=365)
    # Создаем булев индекс для среза датафрейма
    mask = (df[date_col] >= old_start_date) & (df[date_col] <= old_end_date)
    
    # Вырезаем добавочный кусок из данных
    # Применяем булев индекс для получения среза
    result_df = df[mask].copy()

    # Добавляем к data_block_id
    if 'data_block_id' in df.columns:
        result_df['data_block_id'] = result_df['data_block_id'] + 365

    # Добавляем к датам
    for col in add_date_cols:
        result_df[col] = result_df[col] + pd.DateOffset(days=365)
    
    df = pd.concat([df, result_df], ignore_index=True)
    df.to_csv(os.path.join(full_root_path, df_name), index=False)

In [ ]:
# Сборка разделения файлов
def make_full_data():
    enlarge_df('train.csv', 'datetime', ['datetime'])
    enlarge_df('client.csv', 'date', ['date'])
    enlarge_df('gas_prices.csv', 'origin_date', ['origin_date','forecast_date'])
    enlarge_df('electricity_prices.csv', 'origin_date', ['origin_date','forecast_date'])
    enlarge_df('forecast_weather.csv', 'origin_datetime', ['origin_datetime','forecast_datetime'])
    enlarge_df('historical_weather.csv', 'datetime', ['datetime'])

In [ ]:
# Ставим в True, если нужно провести большой тест на 9 месяцев
# и увеличение данных до мая 2024 года

# is_full_test = True
is_full_test = False

if is_full_test and is_local:
    make_full_data()
    root = full_root_path
    train_path = full_train_path
    test_path = full_test_path

In [ ]:
# Разделяет датафрейм на тренировочную и тестовую часть
# Возвращает часть датафрейма для тренировки, тестовую часть датафрейма записывает в каталог с тестами
def split_train_test(filename):
    df = pd.read_csv(os.path.join(root, filename))
    
    #Запишем часть данных для теста
    test_df = df[df["data_block_id"] > train_end_data_block_id]
    if (filename =="train.csv"):
        # Берем только те ячейки где target был не нулевым
        test_df = test_df[test_df["target"].notnull()]
        
    test_df.to_csv(os.path.join(test_path, filename), index=False)

    #Запишем часть данных для трейна
    train_df = df[df["data_block_id"] <= train_end_data_block_id]
    train_df.to_csv(os.path.join(train_path, filename), index=False)

# Доводим до ума тестовые таблицы чтобы они были точно такие как в реальном сабмите
def test_dfs_tune():
    # Делаем таблицу revealed_targets.csv
    df = pd.read_csv(os.path.join(root, "train.csv"))
    df = df[df["data_block_id"] > train_end_data_block_id - 2]
    df["data_block_id"] += 2
    df = df[df["target"].notnull()]
    df.to_csv(os.path.join(test_path, 'revealed_targets.csv'), index=False)
    
    # Делаем таблицу test.csv
    df = pd.read_csv(os.path.join(test_path, "train.csv"))
    df.rename(columns={'datetime': 'prediction_datetime'}, inplace=True)
    df.drop('target', axis=1, inplace=True)

    # По умаолчанию задаем что оцениваться будет весь тестовый датасет
    df['currently_scored'] = True
    
    '''
    # Можно расскомментировать этот блок для проверки локально что работает выделение только части data_block_id,
    # которые будут оцениваться и на которых будем делать предсказание
    df['currently_scored'] = False
    #df.loc[(df['data_block_id'] >= 518) & (df['data_block_id'] <= 525), 'currently_scored'] = True
    df.loc[(df['data_block_id'] >= 518+365) & (df['data_block_id'] <= 606+365), 'currently_scored'] = True
    #df.loc[(df['data_block_id'] >= 690) & (df['data_block_id'] <= 606+365), 'currently_scored'] = True
    '''
    df.to_csv(os.path.join(test_path, 'test.csv'), index=False)
    
    # Делаем таблицу sample_submission.csv
    selected_columns = ['row_id', 'data_block_id']
    df = df[selected_columns]
    df['target'] = 0
    df.to_csv(os.path.join(test_path, 'sample_submission.csv'), index=False)

# Сборка разделения файлов
def make_split():
    # csv файлы которые будем делить:
    csv_names = ["train.csv", "client.csv", "gas_prices.csv", "electricity_prices.csv", "forecast_weather.csv", "historical_weather.csv"]
    for csv_name in csv_names:
        split_train_test(csv_name)
    # Доделываем тестовые таблицы
    test_dfs_tune()

In [ ]:
%%time

# Создаем файлы csv c тренировочными и тестовыми таблицами
if is_local:
    # Пока отключил создание тестовых файлом. У меня локально они есть
    # make_split()
    pass

### Data I/O

In [ ]:
%%time
df_data        = pl.read_csv(os.path.join(train_path, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client      = pl.read_csv(os.path.join(train_path, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas         = pl.read_csv(os.path.join(train_path, "gas_prices.csv"), columns=gas_cols, try_parse_dates=True)
df_electricity = pl.read_csv(os.path.join(train_path, "electricity_prices.csv"), columns=electricity_cols, try_parse_dates=True)
df_forecast    = pl.read_csv(os.path.join(train_path, "forecast_weather.csv"), columns=forecast_cols, try_parse_dates=True)
df_historical  = pl.read_csv(os.path.join(train_path, "historical_weather.csv"), columns=historical_cols, try_parse_dates=True)
#df_location    = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_location    = pl.read_csv('/kaggle/input/locations/county_lon_lats.csv', columns=location_cols, try_parse_dates=True)
df_target      = df_data.select(target_cols)
working_days   = pl.read_csv('/kaggle/input/working-days/working_days.csv', try_parse_dates=True)

schema_data        = df_data.schema
schema_client      = df_client.schema
schema_gas         = df_gas.schema
schema_electricity = df_electricity.schema
schema_forecast    = df_forecast.schema
schema_historical  = df_historical.schema
schema_target      = df_target.schema

### HyperParam Optimization

In [ ]:
# Подготовка данных для поиска гиперпараметров
if is_tuning:
    X, y = df_data.drop("target"), df_data.select("target")
    X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
    df_train = to_pandas(X, y)

    # df_train = df_train[df_train["target"].notnull()].query(training_start_date)
pass

In [ ]:
# Подбор гиперпараметров для модели is_consumption=1
if is_tuning:
    study = optuna.create_study(direction='minimize', study_name='Regressor')
    study.optimize(lgb_objective_cons1, n_trials=100, show_progress_bar=True)
pass

In [ ]:
# Подбор гиперпараметров для модели is_consumption=0
if is_tuning:
    study = optuna.create_study(direction='minimize', study_name='Regressor')
    study.optimize(lgb_objective_cons0, n_trials=100, show_progress_bar=True)
pass

### Validation

In [ ]:
'''result = cross_validate(
    estimator=lgb.LGBMRegressor(**best_params, random_state=42),
    X=df_train.drop(columns=["target"]), 
    y=df_train["target"],
    scoring="neg_mean_absolute_error",
    cv=MonthlyKFold(1),
)

print(f"Fit Time(s): {result['fit_time'].mean():.3f}")
print(f"Score Time(s): {result['score_time'].mean():.3f}")
print(f"Error(MAE): {-result['test_score'].mean():.3f}")'''
pass

### Training

#### Параметры для LGB is_consumption=1

#### Параметры для LGB is_consumption=0

#### Параметры для LGB is_consumption=1

In [ ]:
# Исходные модели
if is_gpu:
    p1={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1569, 'learning_rate': 0.0954350912359592, 'colsample_bytree': 0.5643173334694846, 'colsample_bynode': 0.8507963404594194, 'reg_alpha': 0.5815184353302838, 'reg_lambda': 19.577675580975, 'min_child_samples': 83, 'max_depth': 15, 'max_bin': 125}
    
    p2={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1655, 'learning_rate': 0.09749234861597421, 'colsample_bytree': 0.5878083872493073, 'colsample_bynode': 0.7476463591217907, 'reg_alpha': 0.6253329425224532, 'reg_lambda': 19.31104697178572, 'min_child_samples': 41, 'max_depth': 15, 'max_bin': 116}

    p3={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1702, 'learning_rate': 0.08956547486313553, 'colsample_bytree': 0.553841254939378, 'colsample_bynode': 0.7707977076873439, 'reg_alpha': 3.4503195735482803, 'reg_lambda': 17.09137108374253, 'min_child_samples': 46, 'max_depth': 16, 'max_bin': 123}

    p4={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1662, 'learning_rate': 0.09936096276241443, 'colsample_bytree': 0.5262782304338314, 'colsample_bynode': 0.6861424640373375, 'reg_alpha': 1.1434350179754031, 'reg_lambda': 18.238183112413253, 'min_child_samples': 42, 'max_depth': 16, 'max_bin': 118}

    p5={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1596, 'learning_rate': 0.09145526299958297, 'colsample_bytree': 0.5813223295308532, 'colsample_bynode': 0.8019876742272201, 'reg_alpha': 1.7506271227424264, 'reg_lambda': 19.97101707600914, 'min_child_samples': 45, 'max_depth': 16, 'max_bin': 129}

    p6={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1611, 'learning_rate': 0.09177637954991985, 'colsample_bytree': 0.5406014712979323, 'colsample_bynode': 0.7948709121454631, 'reg_alpha': 1.4162076884265264, 'reg_lambda': 19.18451476080634, 'min_child_samples': 39, 'max_depth': 15, 'max_bin': 128}

    p7={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1722, 'learning_rate': 0.09721924587634734, 'colsample_bytree': 0.581822849292829, 'colsample_bynode': 0.8475502365142857, 'reg_alpha': 0.5558295705320715, 'reg_lambda': 17.676848729848402, 'min_child_samples': 76, 'max_depth': 15, 'max_bin': 121}
    
    p8={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 3080, 'verbose': -1, 'random_state': 49, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.0706777084688013, 'colsample_bytree': 0.8626666525958948, 'colsample_bynode': 0.764542591615078, 'reg_alpha': 17.891843863642414, 'reg_lambda': 13.11440305667691, 'min_child_samples': 13, 'max_depth': -1, 'max_bin': 195}
    p9={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 3080, 'verbose': -1, 'random_state': 50, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.0706777084688013, 'colsample_bytree': 0.8626666525958948, 'colsample_bynode': 0.764542591615078, 'reg_alpha': 17.891843863642414, 'reg_lambda': 13.11440305667691, 'min_child_samples': 13, 'max_depth': -1, 'max_bin': 195}
    p10={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 3080, 'verbose': -1, 'random_state': 51, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.0706777084688013, 'colsample_bytree': 0.8626666525958948, 'colsample_bynode': 0.764542591615078, 'reg_alpha': 17.891843863642414, 'reg_lambda': 13.11440305667691, 'min_child_samples': 13, 'max_depth': -1, 'max_bin': 195}
    p11={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 3080, 'verbose': -1, 'random_state': 52, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.0706777084688013, 'colsample_bytree': 0.8626666525958948, 'colsample_bynode': 0.764542591615078, 'reg_alpha': 17.891843863642414, 'reg_lambda': 13.11440305667691, 'min_child_samples': 13, 'max_depth': -1, 'max_bin': 195}
    p12={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 3080, 'verbose': -1, 'random_state': 53, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.0706777084688013, 'colsample_bytree': 0.8626666525958948, 'colsample_bynode': 0.764542591615078, 'reg_alpha': 17.891843863642414, 'reg_lambda': 13.11440305667691, 'min_child_samples': 13, 'max_depth': -1, 'max_bin': 195}
    p13={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 3080, 'verbose': -1, 'random_state': 54, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.0706777084688013, 'colsample_bytree': 0.8626666525958948, 'colsample_bynode': 0.764542591615078, 'reg_alpha': 17.891843863642414, 'reg_lambda': 13.11440305667691, 'min_child_samples': 13, 'max_depth': -1, 'max_bin': 195}
    p14={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 2133, 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'num_leaves': 48, 'learning_rate': 0.07013300284120705, 'colsample_bytree': 0.9253282723262372, 'colsample_bynode': 0.8346343958070863, 'reg_alpha': 16.241601828339515, 'reg_lambda': 5.350458175997673, 'min_child_samples': 76, 'max_depth': -1, 'max_bin': 162}
else:
    pass

'''
p1={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 2133, 'verbose': -1, 'objective': 'l2', 'num_leaves': 48, 'learning_rate': 0.07013300284120705, 'colsample_bytree': 0.9253282723262372, 'colsample_bynode': 0.8346343958070863, 'reg_alpha': 16.241601828339515, 'reg_lambda': 5.350458175997673, 'min_child_samples': 76, 'max_depth': -1, 'max_bin': 162}
p2={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 2361, 'verbose': -1, 'objective': 'l2', 'num_leaves': 34, 'learning_rate': 0.060949366914939414, 'colsample_bytree': 0.9993288380159902, 'colsample_bynode': 0.9228215866566277, 'reg_alpha': 18.12946199946793, 'reg_lambda': 2.5957584406928143, 'min_child_samples': 85, 'max_depth': -1, 'max_bin': 163}
p3={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 2157, 'verbose': -1, 'objective': 'l2', 'num_leaves': 49, 'learning_rate': 0.072162968496851, 'colsample_bytree': 0.9366582140649763, 'colsample_bynode': 0.9987768911165109, 'reg_alpha': 19.955878587229964, 'reg_lambda': 5.808092072452236, 'min_child_samples': 84, 'max_depth': -1, 'max_bin': 156}
p4={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 2474, 'verbose': -1, 'objective': 'l2', 'num_leaves': 29, 'learning_rate': 0.06309418972836117, 'colsample_bytree': 0.84524749955274, 'colsample_bynode': 0.8613374079014886, 'reg_alpha': 16.597197944229823, 'reg_lambda': 3.226077180374126, 'min_child_samples': 57, 'max_depth': -1, 'max_bin': 133}
p5={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 2279, 'verbose': -1, 'objective': 'l2', 'num_leaves': 26, 'learning_rate': 0.06613865527933853, 'colsample_bytree': 0.9742566223807109, 'colsample_bynode': 0.7969119918576429, 'reg_alpha': 18.7828897848879, 'reg_lambda': 4.246312305249892, 'min_child_samples': 20, 'max_depth': -1, 'max_bin': 156}
p6={'device': 'gpu', 'gpu_platform_id': 1, 'gpu_device_id': 0, 'n_estimators': 2315, 'verbose': -1, 'objective': 'l2', 'num_leaves': 25, 'learning_rate': 0.058945967929007116, 'colsample_bytree': 0.8404315490378437, 'colsample_bynode': 0.7845873376266698, 'reg_alpha': 18.72682046336603, 'reg_lambda': 3.92605604267166, 'min_child_samples': 128, 'max_depth': -1, 'max_bin': 108}
'''
pass

In [ ]:
'''
p1={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1569, 'learning_rate': 0.0954350912359592, 'colsample_bytree': 0.5643173334694846, 'colsample_bynode': 0.8507963404594194, 'reg_alpha': 0.5815184353302838, 'reg_lambda': 19.577675580975, 'min_child_samples': 83, 'max_depth': 15, 'max_bin': 125}
p2={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1569, 'learning_rate': 0.0954350912359592, 'colsample_bytree': 0.5643173334694846, 'colsample_bynode': 0.8507963404594194, 'reg_alpha': 0.5815184353302838, 'reg_lambda': 19.577675580975, 'min_child_samples': 83, 'max_depth': 15, 'max_bin': 125}    
p3={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1655, 'learning_rate': 0.09749234861597421, 'colsample_bytree': 0.5878083872493073, 'colsample_bynode': 0.7476463591217907, 'reg_alpha': 0.6253329425224532, 'reg_lambda': 19.31104697178572, 'min_child_samples': 41, 'max_depth': 15, 'max_bin': 116}
p4={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1655, 'learning_rate': 0.09749234861597421, 'colsample_bytree': 0.5878083872493073, 'colsample_bynode': 0.7476463591217907, 'reg_alpha': 0.6253329425224532, 'reg_lambda': 19.31104697178572, 'min_child_samples': 41, 'max_depth': 15, 'max_bin': 116}
p5={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1702, 'learning_rate': 0.08956547486313553, 'colsample_bytree': 0.553841254939378, 'colsample_bynode': 0.7707977076873439, 'reg_alpha': 3.4503195735482803, 'reg_lambda': 17.09137108374253, 'min_child_samples': 46, 'max_depth': 16, 'max_bin': 123}
p6={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1702, 'learning_rate': 0.08956547486313553, 'colsample_bytree': 0.553841254939378, 'colsample_bynode': 0.7707977076873439, 'reg_alpha': 3.4503195735482803, 'reg_lambda': 17.09137108374253, 'min_child_samples': 46, 'max_depth': 16, 'max_bin': 123}
p7={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1662, 'learning_rate': 0.09936096276241443, 'colsample_bytree': 0.5262782304338314, 'colsample_bynode': 0.6861424640373375, 'reg_alpha': 1.1434350179754031, 'reg_lambda': 18.238183112413253, 'min_child_samples': 42, 'max_depth': 16, 'max_bin': 118}
p8={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1662, 'learning_rate': 0.09936096276241443, 'colsample_bytree': 0.5262782304338314, 'colsample_bynode': 0.6861424640373375, 'reg_alpha': 1.1434350179754031, 'reg_lambda': 18.238183112413253, 'min_child_samples': 42, 'max_depth': 16, 'max_bin': 118}
p9={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1596, 'learning_rate': 0.09145526299958297, 'colsample_bytree': 0.5813223295308532, 'colsample_bynode': 0.8019876742272201, 'reg_alpha': 1.7506271227424264, 'reg_lambda': 19.97101707600914, 'min_child_samples': 45, 'max_depth': 16, 'max_bin': 129}
p10={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1596, 'learning_rate': 0.09145526299958297, 'colsample_bytree': 0.5813223295308532, 'colsample_bynode': 0.8019876742272201, 'reg_alpha': 1.7506271227424264, 'reg_lambda': 19.97101707600914, 'min_child_samples': 45, 'max_depth': 16, 'max_bin': 129}
p11={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1611, 'learning_rate': 0.09177637954991985, 'colsample_bytree': 0.5406014712979323, 'colsample_bynode': 0.7948709121454631, 'reg_alpha': 1.4162076884265264, 'reg_lambda': 19.18451476080634, 'min_child_samples': 39, 'max_depth': 15, 'max_bin': 128}
p12={'device': 'gpu', 'verbose': -1, 'objective': 'l2', 'n_estimators': 1611, 'learning_rate': 0.09177637954991985, 'colsample_bytree': 0.5406014712979323, 'colsample_bynode': 0.7948709121454631, 'reg_alpha': 1.4162076884265264, 'reg_lambda': 19.18451476080634, 'min_child_samples': 39, 'max_depth': 15, 'max_bin': 128}
'''
pass

#### Параметры для LGB is_consumption=0

In [ ]:
# Исходные модели
if is_gpu:
    # Параметры для lgbm c GPU
    c1={'device': 'gpu', 'verbose': -1, 'random_state' : 42, 'objective': 'l2', 'n_estimators': 1961, 'learning_rate': 0.055041729559669385, 'colsample_bytree': 0.8054200071555299, 'colsample_bynode': 0.8827333010526346, 'reg_alpha': 9.649253442752036, 'reg_lambda': 16.98908601233005, 'min_child_samples': 54, 'max_depth': 12, 'max_bin': 36}

    c2={'device': 'gpu', 'verbose': -1, 'random_state' : 43, 'objective': 'l2', 'n_estimators': 1948, 'learning_rate': 0.06233594141892915, 'colsample_bytree': 0.8484245099171761, 'colsample_bynode': 0.899824429438312, 'reg_alpha': 10.7294451589117, 'reg_lambda': 17.69396992827211, 'min_child_samples': 45, 'max_depth': 13, 'max_bin': 32}

    c3={'device': 'gpu', 'verbose': -1, 'random_state' : 44, 'objective': 'l2', 'n_estimators': 1995, 'learning_rate': 0.0633853242094111, 'colsample_bytree': 0.9331894149297775, 'colsample_bynode': 0.972632605889707, 'reg_alpha': 8.983517796023829, 'reg_lambda': 18.03334867391121, 'min_child_samples': 33, 'max_depth': 11, 'max_bin': 48}

    c4={'device': 'gpu', 'verbose': -1, 'random_state' : 45, 'objective': 'l2', 'n_estimators': 1948, 'learning_rate': 0.04980982203453618, 'colsample_bytree': 0.7850607568025258, 'colsample_bynode': 0.9990467893228645, 'reg_alpha': 8.279232611894738, 'reg_lambda': 18.878856721521696, 'min_child_samples': 85, 'max_depth': 12, 'max_bin': 32}

    c5={'device': 'gpu', 'verbose': -1, 'random_state' : 46, 'objective': 'l2', 'n_estimators': 1958, 'learning_rate': 0.06331649649993518, 'colsample_bytree': 0.965107198312526, 'colsample_bynode': 0.9562601410444004, 'reg_alpha': 8.595100697458118, 'reg_lambda': 19.672841466470988, 'min_child_samples': 38, 'max_depth': 14, 'max_bin': 37}

    c6={'device': 'gpu', 'verbose': -1, 'random_state' : 47, 'objective': 'l2', 'n_estimators': 1667, 'learning_rate': 0.06761829944908236, 'colsample_bytree': 0.8491878204722972, 'colsample_bynode': 0.7943301198687824, 'reg_alpha': 12.053887346204482, 'reg_lambda': 17.21503593146002, 'min_child_samples': 48, 'max_depth': 12, 'max_bin': 60}

    c7={'device': 'gpu', 'verbose': -1, 'random_state' : 48, 'objective': 'l2', 'n_estimators': 1965, 'learning_rate': 0.06236463049661708, 'colsample_bytree': 0.9209979630548757, 'colsample_bynode': 0.9718755549743984, 'reg_alpha': 9.485421137815203, 'reg_lambda': 19.728001606564117, 'min_child_samples': 24, 'max_depth': 14, 'max_bin': 35}

    c8={'device': 'gpu', 'n_estimators': 3637, 'verbose': -1, 'random_state': 49, 'objective': 'l2', 'num_leaves': 46, 'learning_rate': 0.03549176718139673, 'colsample_bytree': 0.9642490409419204, 'colsample_bynode': 0.312181596851313, 'reg_alpha': 3.2669304575194196, 'reg_lambda': 16.3440499988092, 'min_child_samples': 25, 'max_depth': -1, 'max_bin': 36}
else:
    pass
pass

#### Функция выделения интервалов

In [ ]:
# Выделяыет из данных указанные интервалы.
# И в каждом интервале выбирает случайно лишь указанную долю данных
# df_train - датафрейм для обработки
# is_consumption какие данные возвращать данные в разрезе is_consumption
# data_block_id_intervals какие данные возвращать данные в разрезе is_consumption
#   это массив масивов. В каждой строке описание периода
#   первая колонка на сколько data_block_id в конеце обучени отстоит от доступного конца данных
#   вторая колонка сколько data_block_id будет в периоде на котором обучаемся.
#   Третья колонка какую долю от данных оставдять (от 0 до 1)
#   Периодов (строк) может быть произвольное количество
# data_block_id_min минмимальный data_block_id который будет в возвращаемых данных
def get_train_intervals(df_train, is_consumption, data_block_id_intervals, data_block_id_min):
    max_block_id = df_train["data_block_id"].max()
    
    df_train_int = df_train[(
        #  выбираем только те данные которые больше data_block_id_min
        (df_train['data_block_id']>=data_block_id_min)
        #  выбираем только те данные для обучения по is_consumption на которых специализируетсмя модель
        &(df_train['is_consumption']==is_consumption)
        # Оставляем только notnull таргеты
        &(df_train["target"].notnull())
    )]

    ind = 0
    for cur_interval in data_block_id_intervals:
        # вырезаем очередной дата блок в указанных границах
        cur_data_block = df_train_int[(
            # До какого data_block_id учим первый блок
            (df_train_int['data_block_id']<=max_block_id-cur_interval[0])
            # C какого data_block_id учим первый блок
            &(df_train_int['data_block_id']>(max_block_id-cur_interval[0]-cur_interval[1]))
        )]
        # print('1. cur_data_block:', cur_data_block['data_block_id'].nunique())
        # print('1.1 cur_data_block:', cur_data_block.shape[0])
        # Берем только часть случайную часть данных из блока
        cur_data_block = cur_data_block.sample(frac = cur_interval[2])
        # print('2. cur_data_block:', cur_data_block['data_block_id'].nunique())
        # print('2.1 cur_data_block:', cur_data_block.shape[0])
        if ind == 0:
            final_train_df = cur_data_block.copy()
        else:
            final_train_df = pd.concat([final_train_df, cur_data_block], ignore_index=True)
        ind += 1
    return(final_train_df)

#### Функция обучения моделей с диска (Быстрая)

In [ ]:
# Обучает модели и данные, записаные на диск
# Быстрая. Отличается тем, что два процесса запускаются и ждут комманд,
# А не завершаются после выполнения обучения модели
# и записывает обученные модели обратно на диск
# models_names - список имен моделей
# models_path - путь на диске где хроанятся модели и данные

def fit_models_from_disk_fast(models_names, models_path):
    # Код процесса, который будет выполнятся параллельно
    process_code = '''
import os
import time
import lightgbm as lgb
import pandas as pd
from joblib import dump
from joblib import load

models_path = 'zzzmodels_pathzzz'

while True:
    model_name = input()
    if model_name == 'exit':
        print(f"Finish")
        break
    
    # Загружаем модель
    model = load(os.path.join(models_path, f'{model_name}.joblib'))
    # Загружаем трейн
    df_train = pd.read_pickle(os.path.join(models_path, f'{model_name}-df_train.pkl'))
    sample_weight = df_train['sample_weight'].values
    # Обучаем модель
    model.fit(
        X=df_train.drop(columns=["target", "data_block_id", "sample_weight"]),
        y=df_train["target"],
        sample_weight=sample_weight
    )
    
    # Обучаем модель
    model.fit(X, y)
    
    dump(model, os.path.join(models_path, f'{model_name}.joblib'))
    
    print(f"Complete: {model_name}")
'''
    process_code = process_code.replace("zzzmodels_pathzzz", models_path)
    process_code1 = process_code
    process_code2 = process_code
    # Запуск дочерних процесса в котором будут тренироваться модели
    # Процесс 1
    process1 = subprocess.Popen(['python', '-c', process_code1],
                                stdin=subprocess.PIPE, 
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                text=True)
    # Процесс 2
    process2 = subprocess.Popen(['python', '-c', process_code2],
                                stdin=subprocess.PIPE, 
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                text=True)
    # Перебираем все модели, которые будем тренировать
    # По две сразу. Если моделей нечетное model_name2 в последней итерации будет Nono
    for model_name1, model_name2 in zip_longest(
        models_names[::2], models_names[1::2], fillvalue=None):

        print(p_time(), 'fit model:', model_name1)
        #print(p_time(),'Start1.1')
        # Отправка команды обучать модель дочернему процессу1
        process1.stdin.write(f'{model_name1}\n')
        process1.stdin.flush()

        if model_name2 is not None:
            print(p_time(), 'fit model:', model_name2)
            #print(p_time(),'Start1.2')
            # Отправка команды обучать модель дочернему процессу2
            process2.stdin.write(f'{model_name2}\n')
            process2.stdin.flush()
        '''
        # Проверка ошибки выполанния дочернего процесса
        # **************************
        process1.wait()
        # Вывод результатов
        output1, error1 = process1.communicate()
        if output1:
            print("Вывод программы 1:")
            print(output1)
        if error1:
            print("Ошибка выполнения программы 1:")
            print(error1)
        # **************************
        '''
        #print(p_time(),'Start2')
        # Ждем сообщения о завершенеии тренировки модели от дочерних процессов
        response1 = process1.stdout.readline().strip()
        if model_name2 is not None:
            response2 = process2.stdout.readline().strip()
        #print(p_time(),'Start3')
        
        print(p_time(),f"Дочерний процесс1: {response1.strip()}")
        if model_name2 is not None:
            print(p_time(),f"Дочерний процесс2: {response2.strip()}")
        
    # Отправка команды завершения
    #response, _ = process.communicate(input='exit\n')
    process1.stdin.write('exit\n')
    process1.stdin.flush()
    if model_name2 is not None:
        process2.stdin.write('exit\n')
        process2.stdin.flush()
    
    # Ожидание завершения дочернего процесса
    #print(p_time(),'Start4')
    process1.wait()
    if model_name2 is not None:
        process2.wait()
    #print(p_time(),'Start5')
    
    print(p_time(), "Обучение моделей параллельно завершено")

#### Функция обучения-предсказания моделей c диска

In [ ]:
# Возвращает строку с дочерним процессом для обучения и предсказания указанных моделей
# models_names_to_train - список имен моделей, которые нужно тренировать
# models_names_to_test - список имен моделей, которые нужно тестировать
# models_path - путь на диске где хроанятся модели и данные
# gpu_device_id - Номер графического процессора на котором будет выполнятся

def get_proc_code(models_names_to_train, models_names_to_test,
                  models_path, drop_cols, is_consumption, gpu_device_id):
    process_code = f'''
import os
import time
import lightgbm as lgb
import pandas as pd
import numpy as np
from joblib import dump
from joblib import load

process_starttime = time.time()

# Возвращает сколько уже работает процесс
def p_time():
    run_time = round(time.time() - process_starttime)
    return '    '+str(run_time).zfill(5)+' sec:'
    
models_path = '{models_path}'
models_names_to_train = {models_names_to_train}
models_names_to_test = {models_names_to_test}
drop_cols = {drop_cols}
is_consumption = {is_consumption}

# Загружаем данные для предсказания
X_test = pd.read_pickle(os.path.join(models_path, f'x_test.pkl'))
# Если берутся одинаково датаблоки дропов от первой модели для всех
# df_train = pd.read_pickle(os.path.join(models_path, 'df_train.pkl'))

# Перебираем модели, с помощью которых нужно сделать предсказания
# в этом дочернем процесс
for model_name in models_names_to_test:
    # Загружаем модель
    model = load(os.path.join(models_path, model_name+'.joblib'))
    model.set_params(device='{gpu_type}',
                     n_jobs={n_jobs},
                     gpu_platform_id=0,
                     gpu_device_id={gpu_device_id})
    model_drop_cols = drop_cols[model_name] + ['data_block_id']
    model_cons = is_consumption[model_name]
    if model_name in models_names_to_train:
        # Если модель в списке для тренировки
        # Загружаем трейн
        print(p_time(), 'fit model:', model_name)
        df_train = pd.read_pickle(os.path.join(models_path, model_name+'-df_train.pkl'))
        # Обучаем модель
        model.fit(
            # Если берутся одинаково датаблоки дропов от первой модели для всех
            # X=df_train.drop(columns=model_drop_cols+['target']),
            X=df_train.drop(columns=["target", "data_block_id"]),
            y=df_train["target"]
            # Если берутся одинаково датаблоки дропов от первой модели для всех
            # X=df_train[df_train['is_consumption']==model_cons].drop(columns=model_drop_cols+['target']),
            # y=df_train[df_train['is_consumption']==model_cons]["target"]
            # для предсказаний дифов
            # y=df_train["target"] - df_train["target_1"].fillna(0)
        )
        dump(model, os.path.join(models_path, model_name+'.joblib'))
    
    # Делаем новый предикт
    print(p_time(), 'predict model:', model_name)
    # print('model_drop_cols:', model_drop_cols)
    y_pred = model.predict(X_test.drop(columns=model_drop_cols)).clip(0)\
    # для предсказаний дифов
    #y_pred = np.clip(
    #    X_test["target_mean_1"].fillna(0).values
    #    + model.predict(X_test.drop(columns=model_drop_cols)),
    #    0,
    #    np.inf
    #)
    # Сохранение предсказаний в файл
    np.save(os.path.join(models_path, model_name+'-y_pred'), y_pred, allow_pickle=True)
    
print(p_time(), f"Complete")
'''
    return process_code

In [ ]:
# Обучает модели и данные, записаные на диск
# Простая. Отличается тем, что процесс просто запускается,
# сразу обучают модель и завершает работу
# и записывает обученные модели обратно на диск
# models_names_to_train - список имен моделей, которые нужно тренировать
# models_names_to_test - список имен моделей, которые нужно тестировать
# models_path - путь на диске где хроанятся модели и данные

def fit_predict_models_from_disk(
    models_names_to_train, models_names_to_test, models_path, drop_cols, is_consumption):
    print(p_time(), "Обучение моделей параллельно")
    
    # Сортируем models_names_to_test, чтобы сначала в нем шли модели из models_names_to_train
    # Чтобы равномерно распределить модели для тренировки между процессами
    models_names_to_test = sorted(
        models_names_to_test,
        key=lambda x: (
            x not in models_names_to_train,
            models_names_to_train.index(x)
            if x in models_names_to_train
            else float('inf')))
    
    # Разделение моделей для трейна на группы для каждого процесса
    model_groups = [models_names_to_test[i::num_processes] for i in range(num_processes)]

    processes = []

    for process_id, model_group in enumerate(model_groups):
        # Получаем код программы для процесса
        proc_code = get_proc_code(
            models_names_to_train=models_names_to_train,
            models_names_to_test=model_group,
            models_path=models_path,
            drop_cols=drop_cols,
            is_consumption=is_consumption,
            gpu_device_id=(process_id % gpus_n))

        # Записываем код в файл
        proc_filename = os.path.join(models_path, f'process_{process_id}.py')
        with open(proc_filename, 'w', encoding='utf-8') as file:
            file.write(proc_code)
            
        process = subprocess.Popen(
            ['python', proc_filename],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        processes.append(process)
        # print('process_id:',process_id)
        # print((process_id % gpus_n))

    # Выставим process_terminated в True, если прерывали процесс
    process_terminated = False
    # Ожидание завершения всех процессов
    for idx, process in enumerate(processes):
        # Время сколько ждем максимально заверщения процесса
        wait_time = max((stop_train_time - time.time()), 1)
        if (is_disable_run_time_limit or (not models_names_to_train)):
            # Если отключили таймаут или не обучаем модели просто ждем процесс
            process.wait()
        else:
            try:
                process.wait(timeout=wait_time)
            except subprocess.TimeoutExpired:
                # Если время ожидания превышено, принудительно завершаем процесс
                process_terminated = True
                process.terminate()

        if not process_terminated:
            output, error = process.communicate()
            if output:
                print(f"Дочерний процесс {idx}:")
                print(output)
            if is_local and error:
                print(f"Ошибка выполнения программы {idx}:")
                print(error)
        else:
            print(p_time(), f"Прервали принудительно процесс {idx} обучение моделей")

    if process_terminated:
        # Заново запускаем предсказания но уже без тренировки моделей
        # если было превышено время для тренировки моделей
        fit_predict_models_from_disk([], models_names_to_test, models_path, drop_cols, is_consumption)
    
    print(p_time(), "Обучение моделей параллельно завершено")

In [ ]:
# Стрелочник. Выбирает какое обучение запустить быстрое или простое
# models_names - список имен моделей
# models_path - путь на диске где хроанятся модели и данные
def fit_models_from_disk(models_names, models_path):
    #fit_models_from_disk_simple(models_names, models_path)
    fit_models_from_disk_fast(models_names, models_path)

#### Класс моделей

In [ ]:
#***

# Класс обертка для моделей. Хранит различные пареметры для обучения моделей.
# Например диапазоны данных на которых учить модель, как часто обучать заново и другие параметры
class Models:

    # Путь где хранятся модели, тренировочные и тестовые выборки для них 
    models_path = 'models'
    
    
    # Инициализирует параметры обучения
    # init_model - готовый объект модели для обучения
    # alphas_* - Массивы с коэффициентами, на которые домножают предсказания
    # alphas_1 - массив для моделей с предсказанием is_consumption == 1
    # alphas_0 - массив для моделей с предсказанием is_consumption == 0
    def __init__(self, alphas_1, alphas_0):
        # Инициализируем словари
        # Ключами во всех словарях будет имя модели

        # Имена моделей
        self.model_names = []
        
        # Словарь с моделями
        self.models =  dict()
        
        # Словарь с описанием периодов обучения модели
        # пока это матрица. из двух столбцов и двух строк в каждой строке описание периода
        # первая колонка на сколько data_block_id в конеце обучени отстоит от доступного конца данных
        # вторая колонка сколько data_block_id будет в периоде на котором обучаемся.
        # data_block_id могут быть эквивалентны дням, но могут и отличаться, если данные будут подавать блоками не равными дням
        # либо если будут разрывы в данных. Но они точно будут эквиваленты циклам предсказания
        self.data_block_id_intervals  = dict()

        # Словарь с указанием по какой минимальный data_block_id отрезать данные.
        # То есть меньше data_block_id_min не берем данные для обучения в любом случае, чтобы не было определено в data_block_id_intervals
        self.data_block_id_min  = dict()
        
        # Если 1, то модель предназначена для предсказания потребления электричества
        # Если 0, то модель предназначена для предсказания производства электричества
        self.is_consumption = dict()
        
        # Раз во сколько иттераций обучат модель
        self.learn_again_period = dict()
        
        # Смещение для начала обучения модели. Добавляется к номеру итерации сабмита.
        # Скажем если смещение 6 номер итерации 1, а обучаемся раз в чем итераций. То обучение будет в первуже итерацию сабмита.
        self.learn_again_offset = dict()

        # Признаки которые нужно дропнуть у модели
        self.drop_cols = dict()
        
        # Время в секундах сколько заняло последнее обучение модели
        self.last_learn_time = dict()

        # Обучена ли модель
        self.is_trained = dict()

        # Предсказания модели
        self.predictions = dict()
        
        # Коэффициент на который умножать предсказания модели
        self.predictions_alpha = dict()
        
        #Номер итерации. Обновляется при вызове метода fit
        self.itter_n = 0
        
        # Массивы с коэффициентами, на которые домножают предсказания
        # В нулевом индексе массива если устаревание 0 дней, в 1 если 1 день устарела и так далее
        # self.alphas = [41/240, 39/240]
        self.alphas_1 = alphas_1
        self.alphas_0 = alphas_0
        
        # Устанавливаем в True если будем возвращать сохраненные ранее предикты
        self.is_saved_predict = False

        if not os.path.exists(self.models_path):
            # Создание каталога, если его нет
            os.makedirs(self.models_path)
    
    
    # Добавляет еще одну модель
    # model_name - название модели
    # new_model - объект модели
    def add_model(self, model_name, new_model, is_consumption, data_block_id_intervals,
                  data_block_id_min, learn_again_period, learn_again_offset, drop_cols = []):
        
        self.model_names.append(model_name)
        self.models[model_name] = new_model
        self.is_consumption[model_name] = is_consumption
        self.data_block_id_intervals[model_name] = data_block_id_intervals
        self.data_block_id_min[model_name] = data_block_id_min
        self.learn_again_period[model_name] = learn_again_period
        self.learn_again_offset[model_name] = learn_again_offset
        self.drop_cols[model_name] = drop_cols
        self.last_learn_time[model_name] = 0
        self.is_trained[model_name] = False
        self.predictions[model_name] = []
        self.predictions_alpha[model_name] = []

        # Сохранаяем начальную модель на диске
        dump(new_model, os.path.join(self.models_path, f'{model_name}.joblib'))
        
        
    # Обучает модель
    # model_name - название модели
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    def fit_one_model(self, model_name, df_train):
        #print(p_time(), 'fit model:', model_name)
        '''
        #print(p_time(), 'fit3')
        sample_weight = df_train['sample_weight'].values
        self.models[model_name].fit(
            X=df_train.drop(columns=["target", "data_block_id", "sample_weight"]),
            y=df_train["target"],
            sample_weight=sample_weight
        )
        #print(p_time(), 'fit5')
        '''
        #df_train.to_pickle(os.path.join(self.models_path, f'{model_name}-df_train.pkl'))
        pass
    
    
    # записывает коэффициент альфа на который домножать предсказание
    # model_name - название модели
    def make_alpha(self, model_name):
        if (((self.itter_n + self.learn_again_offset[model_name])
             % self.learn_again_period[model_name]) == 0):
            # Если модель только что училась ставим коэффициент первый, поменьше
            self.predictions_alpha[model_name] = (self.alphas_0[0]
                                                  if self.is_consumption[model_name] == 0
                                                  else self.alphas_1[0])
        else:
            # Если модель училась в прошлой итерации
            # ставим коэффициент второй, побольше
            self.predictions_alpha[model_name] = (self.alphas_0[1]
                                                  if self.is_consumption[model_name] == 0
                                                  else self.alphas_1[1])
                                                  
    '''
    # записывает коэффициент альфа на который домножать предсказание
    # model_name - название модели
    def make_alpha(self, model_name):
        
        self.predictions_alpha[model_name] = (1/2 if model_name == 'model-1' else 1/6)
    '''        
    
    # Инициализирует следующую итерацию обучения и предсказания
    # itter_n - номер иттерации в сабмите.
    def init_iter(self, itter_n):
        self.itter_n = itter_n
        # В этот списко будем заносить модели, которые нужно тренировать
        self.model_names_to_train = []        
    
    # Обучает все добавленные модели для которых наступило время их обучения
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    # если itter_n равен <=0. Значит первоначальное обучение и обучаем всем модели
    def fit(self, df_train):
        if self.is_saved_predict:
            # Не обучаем модели, если в режиме сохраненных предсказаний
            return()
            
        max_block_id = df_train["data_block_id"].max()
     
        # Перебираем все модели
        for model_name in self.learn_again_period:

            # Проверяем не нужно ли прервать запись выборок для обучения по таймауту
            if ((time.time() > stop_train_time)
                and not(is_disable_run_time_limit)):
                # Нужно прерывать обучение моделей
                print(p_time(), 'Прерываем обучение моделей из цикла записи тренировочных выборок')
                self.model_names_to_train = []
                break
            # Либо модель еще не тренирована
            if ((not self.is_trained[model_name])
                # Либо учим если номер итераиции в сабмите плюс смещение делится на целое на период обучения
                or (((self.itter_n + self.learn_again_offset[model_name])
                     % self.learn_again_period[model_name]) == 0)):

                df_train_int = df_train[(
                    #  выбираем только те данные которые больше data_block_id_min
                    (df_train['data_block_id']>=self.data_block_id_min[model_name])
                    #  выбираем только те данные для обучения по is_consumption на которых специализируетсмя модель
                    &(df_train['is_consumption']==self.is_consumption[model_name])
                    # Оставляем только notnull таргеты
                    &(df_train["target"].notnull())
                )]

                ind = 0
                for cur_interval in self.data_block_id_intervals[model_name]:
                    # вырезаем очередной дата блок в указанных границах
                    cur_data_block = df_train_int[(
                        # До какого data_block_id учим первый блок
                        (df_train_int['data_block_id']<=max_block_id-cur_interval[0])
                        # C какого data_block_id учим первый блок
                        &(df_train_int['data_block_id']>(max_block_id-cur_interval[0]-cur_interval[1]))
                    )]
                    # print('1. cur_data_block:', cur_data_block['data_block_id'].nunique())
                    # print('1.1 cur_data_block:', cur_data_block.shape[0])
                    # Берем только часть случайную часть данных из блока
                    cur_data_block = cur_data_block.sample(frac = cur_interval[2])
                    # print('2. cur_data_block:', cur_data_block['data_block_id'].nunique())
                    # print('2.1 cur_data_block:', cur_data_block.shape[0])
                    if ind == 0:
                        final_train_df = cur_data_block.copy()
                    else:
                        final_train_df = pd.concat([final_train_df, cur_data_block], ignore_index=True)
                    ind += 1

                '''
                print('df_train_int:', df_train_int['data_block_id'].nunique())
                print('df_train_int:', df_train_int.shape[0])
                print('df_train:', df_train['data_block_id'].nunique())
                print('df_train:', df_train.shape[0])
                print('final_train_df:', final_train_df['data_block_id'].nunique())
                print('final_train_df:', final_train_df.shape[0])
                '''

                # print('zzz:', self.drop_cols[model_name])
                # Удаляем признаки, которые были указаны к удалению для текущей модели
                final_train_df = final_train_df.drop(columns=self.drop_cols[model_name])

                # Сохраняем трейн для дальнейшего обучения параллельными процессами
                final_train_df.to_pickle(os.path.join(self.models_path, f'{model_name}-df_train.pkl'))
                
                self.model_names_to_train.append(model_name)
                # Отмечаем что модель тренирована (по крайней мере будет тренирована)
                self.is_trained[model_name] = True
                
    
    # Делает предсказание всеми добавленными моделями и сводит их в одно предсказание
    def predict(self, X_test):
        # Создаем датафрейм для предсказаний
        # Первый столбец содержит информацию 'is_consumption'
        # Для каждой модели отдельный столбец с предсказаниями
        # В predict_df_0 будут предсказания для моделей с is_consumption == 0
        self.predict_df_0 = pd.DataFrame(X_test['is_consumption'])
        # В predict_df_1 будут предсказания для моделей с is_consumption == 1
        self.predict_df_1 = pd.DataFrame(X_test['is_consumption'])
        
        # Сохраняем X_test для дальнейшего предсказания по нему параллельными процессами
        X_test.to_pickle(os.path.join(self.models_path, f'x_test.pkl'))
        
        # Обучаем модели, записаные на диск и делаем предсказание
        fit_predict_models_from_disk(
            models_names_to_train = self.model_names_to_train,
            models_names_to_test = list(self.models.keys()),
            models_path = self.models_path,
            drop_cols = self.drop_cols,
            is_consumption = self.is_consumption
        )
        
        # Перебираем все модели и записываем что предсказать на диск
        for model_name in self.learn_again_period:
            self.make_alpha(model_name)
            
        # Перебираем все модели и загружаем предсказания
        for model_name in self.learn_again_period:
            # Делаем предсказание в соответвующий датафрейм в столбец модели
            y_pred = np.load(os.path.join(self.models_path, f'{model_name}-y_pred.npy'), allow_pickle=True)
            self.predictions[model_name].append(y_pred)
            
            if self.is_consumption[model_name] == 0: 
                self.predict_df_0[model_name] = y_pred
                # Домножаем предсказание на коэффициент
                self.predict_df_0[model_name] = self.predict_df_0[model_name] * self.predictions_alpha[model_name]
            else:
                self.predict_df_1[model_name] = y_pred
                self.predict_df_1[model_name] = self.predict_df_1[model_name] * self.predictions_alpha[model_name]

        # Суммируем предсказания моделей раздельно по датафреймам
        self.predict_df_0['target'] = self.predict_df_0.iloc[:, 1:].sum(axis=1) #mean(axis=1)
        self.predict_df_1['target'] = self.predict_df_1.iloc[:, 1:].sum(axis=1) #mean(axis=1)
        
        #self.predict_df_0['target'] = self.predict_df_0.iloc[:, 1:].mean(axis=1)
        #self.predict_df_1['target'] = self.predict_df_1.iloc[:, 1:].mean(axis=1)
        
        # Сведение в одно предсказание потребления и производства электричества у просьюмеров
        predict_df = self.predict_df_1[['is_consumption', 'target']]

        predict_df.loc[predict_df['is_consumption']==0, 'target'] = self.predict_df_0.loc[self.predict_df_0['is_consumption']==0, 'target']
        
        return predict_df['target'].values


    # Записывает в модели пустые предсказания
    def dummy_predict(self, y_pred):
        for model_name in self.learn_again_period:
            # Записываем пустые предсказания в модели
            self.predictions[model_name].append(y_pred)

In [ ]:
#tresk = np.load(os.path.join('models', f'model-1-y_pred.npy'), allow_pickle=True)
#tresk.shape[0]

#### Добавление моделей

In [ ]:
# Для удаления созданных фич для удобстава создаем датафрей и в нем ищем колонки
# Но для сокорости можно просто задать список (отрабатывает за 11 секунд один раз в самом начале)
X, y = df_data.drop("target"), df_data.select("target")
X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
df_train = to_pandas(X, y)

In [ ]:
X.shape

In [ ]:
# Возвращает список к удалению всех погодных лагов
# кроме тех что сдыинуты на указанный день
# lag_day - день сдвига, который не нужно удалять из лагов
def get_one_weather_lag(lag_day):
    wheather_lag_cols = []
    for i in [1,2,3,4,7]:
        wheather_lag_cols.extend(
            [col for col in df_train.columns
             if (col.endswith((f'_fd{i}', f'_fl{i}', f'_hd{i}', f'_hl{i}')))
                 and not col.endswith((f'_fd{lag_day}', f'_fl{lag_day}', f'_hd{lag_day}', f'_hl{lag_day}'))])
    
    return wheather_lag_cols

# Возвращает список к удалению всех погодных лагов
# кроме тех что сдыинуты на указанный день
# lag_day - день сдвига, который не нужно удалять из лагов
def get_one_hour_weather_lag(lag_hour):
    wheather_lag_cols = []
    for i in [1,2]:
        wheather_lag_cols.extend(
            [col for col in df_train.columns
             if (col.endswith((f'_forecast_{i}h', f'_forecast_local_{i}h')))
                 and not col.endswith((f'_forecast_{lag_hour}h', f'_forecast_local_{lag_hour}h'))])
    
    return wheather_lag_cols

# Удаляет элементы из списка
# my_list список из которого удаляем эдлементы
# strings_to_remove список строк которые нужно удалить из my_list
def remove_list_elements(my_list, strings_to_remove):
    for string in strings_to_remove:
        if string in my_list:
            my_list.remove(string)
    return my_list

In [ ]:
%%time
cons1_drop_cols = ['target_ratio_y_1','target_mean_y_1','target_std_y_1',
                   'target_y_1','target_y_2','target_y_3','target_y_4',
                   'target_y_5','target_y_6','target_y_7','target_y_8',
                   'target_std_2','target_mean_2','target_ratio_2',
                   #'direct_solar_radiation','diffuse_radiation'
                  ]
# Для cons1 удаляем все погодные лаги кроме как на неделю как было и раньше

cons1_drop_cols.extend(get_one_hour_weather_lag(0))
cons1_drop_cols.extend(get_one_weather_lag(7))
cons1_drop_cols = remove_list_elements(
    cons1_drop_cols,
    ['hours_ahead_fd7',
     'temperature_fd7',
     'hours_ahead_fl7',
     'temperature_fl7',
     'temperature_hd7',
     'temperature_hl7',
     #'rad_sum',
     #'rad_sum_hl',
    ]
)
cons0_drop_cols = []
cons0_drop_cols.extend(get_one_weather_lag(0))

block = [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]]
#block_cons1 = [[0,30,1],[30,70,0.7],[100,100,0.25],[200,180,0.15],[380,1000,0.1]]
block_cons1 = [[0,100,1],[100,100,0.5],[200,180,0.3],[380,1000,0.2]]
# block = [[0,10000,1]]


models_list = [
    {'model_name': 'model-1', 'new_model': lgb.LGBMRegressor(**p1), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    {'model_name': 'model-2', 'new_model': lgb.LGBMRegressor(**p2), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    {'model_name': 'model-3', 'new_model': lgb.LGBMRegressor(**p3), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    {'model_name': 'model-4', 'new_model': lgb.LGBMRegressor(**p4), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    {'model_name': 'model-5', 'new_model': lgb.LGBMRegressor(**p5), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    {'model_name': 'model-6', 'new_model': lgb.LGBMRegressor(**p6), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    #{'model_name': 'model-7', 'new_model': lgb.LGBMRegressor(**p7), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    #{'model_name': 'model-8', 'new_model': lgb.LGBMRegressor(**p8), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    #{'model_name': 'model-9', 'new_model': lgb.LGBMRegressor(**p9), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    #{'model_name': 'model-10', 'new_model': lgb.LGBMRegressor(**p10), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    #{'model_name': 'model-11', 'new_model': lgb.LGBMRegressor(**p11), 'learn_again_period': 2, 'learn_again_offset': 0, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    #{'model_name': 'model-12', 'new_model': lgb.LGBMRegressor(**p12), 'learn_again_period': 2, 'learn_again_offset': 1, 'is_consumption': 1, 'data_block_id_intervals': block_cons1, 'data_block_id_min': 0, 'drop_cols': cons1_drop_cols},
    {'model_name': 'model-solar-1', 'new_model': lgb.LGBMRegressor(**c1), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    {'model_name': 'model-solar-4', 'new_model': lgb.LGBMRegressor(**c2), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    {'model_name': 'model-solar-2', 'new_model': lgb.LGBMRegressor(**c3), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    {'model_name': 'model-solar-3', 'new_model': lgb.LGBMRegressor(**c4), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    {'model_name': 'model-solar-5', 'new_model': lgb.LGBMRegressor(**c5), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    {'model_name': 'model-solar-6', 'new_model': lgb.LGBMRegressor(**c6), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-7', 'new_model': lgb.LGBMRegressor(**c7), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-8', 'new_model': lgb.LGBMRegressor(**c8), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-9', 'new_model': lgb.LGBMRegressor(**c9), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-10', 'new_model': lgb.LGBMRegressor(**c10), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-11', 'new_model': lgb.LGBMRegressor(**c11), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-12', 'new_model': lgb.LGBMRegressor(**c12), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-7', 'new_model': lgb.LGBMRegressor(**c7), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-8', 'new_model': lgb.LGBMRegressor(**c8), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-9', 'new_model': lgb.LGBMRegressor(**c9), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-10', 'new_model': lgb.LGBMRegressor(**c10), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-11', 'new_model': lgb.LGBMRegressor(**c11), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
    #{'model_name': 'model-solar-12', 'new_model': lgb.LGBMRegressor(**c12), 'learn_again_period': 1, 'learn_again_offset': 0, 'is_consumption': 0, 'data_block_id_intervals': block, 'data_block_id_min': 0, 'drop_cols': cons0_drop_cols},
]

pass

##### Вариант 5

In [ ]:
is_cons_1_n = sum(model['is_consumption'] == 1 for model in models_list)
is_cons_0_n = sum(model['is_consumption'] == 0 for model in models_list)

#print("Number of rows with is_consumption == 1:", is_cons_1_n)
#print("Number of rows with is_consumption == 0:", is_cons_0_n)

# Вариант с одинаковыми коэффцициентами и подстройкой под число моделей в списке
# А вообще это коэффиценты для предсказания моделей обученный в этой итерации и в прошлой
models = Models(alphas_1=[1/is_cons_1_n, 1/is_cons_1_n], alphas_0=[1/is_cons_0_n, 1/is_cons_0_n])

In [ ]:
# Добавляем модели в объект класса моделей
for model_param in models_list:
    models.add_model(**model_param)

#### Обучение моделей

In [ ]:
#if is_local:
#    dump(model_solar, 'model_solar.joblib')
#    dump(model, 'model_lgbm.joblib')

Расскоментировать при необходимости загрузку ранее сохраненных моделей

In [ ]:
# Для загрузки локально
#model_solar = load('model_solar.joblib')
#model = load('model_lgbm.joblib')

# Для загрузки на kaggle
#model_solar = load('/kaggle/input/enefit/model_solar.joblib')
#model = load('/kaggle/input/enefit/model_lgbm.joblib')

## Prediction

In [ ]:
if is_local:
    # Если выполняем локально, а не сабмитим на кагл,
    # то выбираем другое имя для файла submission.csv.
    # Потому что в submission.csv записать прав нет и вылетает по ошибке
    submission_name = 'submission_loc.csv'
else:
    submission_name = 'submission.csv'

### Содержимое public_timeseries_testing_util.py

С необходимыми праками. Решил не импортировать его. а прямо тут. Так удобнее переносить на kaggle

In [ ]:
'''
An unlocked version of the timeseries API intended for testing alternate inputs.
Mirrors the production timeseries API in the crucial respects, but won't be as fast.

ONLY works afer the first three variables in MockAPI.__init__ are populated.
'''

from typing import Sequence, Tuple


class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = [f'{test_path}/test.csv',
                                   f'{test_path}/revealed_targets.csv', 
                                   f'{test_path}/client.csv',
                                   f'{test_path}/historical_weather.csv',
                                   f'{test_path}/forecast_weather.csv',
                                   f'{test_path}/electricity_prices.csv',
                                   f'{test_path}/gas_prices.csv',
                                   f'{test_path}/sample_submission.csv']
        self.group_id_column: str = 'data_block_id'
        self.export_group_id_column: bool = False
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open(submission_name, 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()


### Функция для скора

In [ ]:
# Вычисляет скор для предсказаний, который были поданы на вход
# compare - датафрейм с уже заполенными реальными значениями таргета
# и сделанными предсказаниями

def compute_compare(compare, index_name):
    mae = mean_absolute_error(compare['target'] , compare['predict'])

    if (compare['data_block_id'].max() >= 518):
        compare_temp = compare[(compare['data_block_id'] >= 518)&(compare['data_block_id'] <= 606)]
        mae_518_606 = mean_absolute_error(compare_temp['target'], compare_temp['predict'])
    else:
        # Если еще не дошли до data_block_id >= 518 не считаем эти величины
        mae_518_606 = '-'
    
    if (compare['data_block_id'].max() > 600):
        # Считаем MAE для data_block_id > 600
        compare_temp = compare[compare['data_block_id'] > 600]
        mae_600 = mean_absolute_error(compare_temp['target'], compare_temp['predict'])
    else:
        # Если еще не дошли до data_block_id > 600 не считаем эти величины
        mae_600 = '-'

    mae_df = pd.DataFrame({
        '(ALL)': mae,
        'Feb - Apr (518 - 606)': mae_518_606,
        '(> 600)': mae_600
    }, index=[index_name])

    # Округляем числа до двух знаков после запятой и преобразуем их в строки
    mae_df = mae_df.round(3).astype(str)
    
    return mae_df

In [ ]:
# Подсчитывает скор.
# Возвращает датафрейм compare со сравнением предсказаний
def calc_score():
    # Загружаем предсказания
    #submission = pd.read_csv(submission_name)
    submission = pd.concat(env.predictions)
    
    # Загружаем истинные значения
    revealed_targets = pd.read_csv(os.path.join(test_path, "revealed_targets.csv"))
    revealed_targets['data_block_id'] -= 2
    revealed_targets = revealed_targets[revealed_targets["data_block_id"] > train_end_data_block_id]
    # Обрезаем реальные предсказания revealed_targets по длине уже сделанных предсказаний submission
    revealed_targets = revealed_targets.iloc[:len(submission)]

    # print(f'MAE: {mae}')
    
    # Подготовим данные для анализа изменения ошибки предсказания по мере удаления от времени завершения обучения
    compare = revealed_targets[['data_block_id', 'is_consumption', 'target']].copy()
    compare['predict'] = submission['target'].values
    compare['abs_err'] = abs(compare['predict'] - compare['target']).values
    compare['err'] = (compare['predict'] - compare['target']).values

    mae_df = compute_compare(compare, 'All Models MAE')

    new_mae_df = compute_compare(compare[compare['is_consumption']==0], 'is_consumption == 0')
    mae_df = pd.concat([mae_df, new_mae_df])

    new_mae_df = compute_compare(compare[compare['is_consumption']==1], 'is_consumption == 1')
    mae_df = pd.concat([mae_df, new_mae_df])

    # Перебираем предсказания всех моделей и складываем
    # скоры их предсказания в один датафрейм mae_df
    for model_name in models.predictions:
        # заполняем compareпредсказаниями модели
        compare['predict'] = np.concatenate(models.predictions[model_name])
        compare['abs_err'] = abs(compare['predict'] - compare['target']).values
        compare['err'] = (compare['predict'] - compare['target']).values
        # Оставляем в предсказаниях только те строки, которые предсказывала модель
        new_compare = compare[compare['is_consumption']==models.is_consumption[model_name]] 
        
        new_mae_df = compute_compare(new_compare,
                                     model_name+f' ({models.is_consumption[model_name]})')
                                                 
        mae_df = pd.concat([mae_df, new_mae_df])
    
    display(mae_df)
    print('MAE > 600', mae_df.loc['All Models MAE','(> 600)'])

    compare['predict'] = submission['target'].values
    compare['abs_err'] = abs(compare['predict'] - compare['target']).values
    compare['err'] = (compare['predict'] - compare['target']).values
    
    return compare

### Инициализация иттераций сабмита

In [ ]:
if is_local:
    # После этого можно имитировать локально загрузку при собмите на большом числе итераций
    # А не только четыре иттерации на 4 дня как в стандартной имитайии на кагле
    env = make_env()
else:
    # загружаем оригинальную библиотеку для сабмита
    import enefit
    env = enefit.make_env()

iter_test = env.iter_test()

### Цикл сабмита

In [ ]:
# Находим последний data_block_id в обучающих данных
max_train_data_block_id = df_data["data_block_id"].max()
# Устанавливаем первый data_block_id для теста следущим за тренировочным
cur_test_data_block_id = max_train_data_block_id + 1
cur_test_data_block_id

In [ ]:
%%time
count = 0

# Основной цикл для обработки данных тестового набора
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    iteration_start_time = time.time()
    print(p_time(), f'*************** Iteration: {count}, data_block_id: {cur_test_data_block_id-1} ***************')
    # iteration__scrored - устанавливается в True, если эту итерацию нужно предсказывать
    iteration__scrored = (not (test['currently_scored'] == False).all())
    
    # Переименование столбца для удобства
    test = test.rename(columns={"prediction_datetime": "datetime"})
        
    if is_local:
        # Если выполняем локально, то преобразуем некоторые типы данных
        # На кагле (а может и в линуксе) они и так преобразуются, но на виновс локально
        # не преобразуются и выдетают по ощибке
        test['datetime'] = pd.to_datetime(test['datetime'])
        client['date'] = pd.to_datetime(client['date'])
        gas_prices['origin_date'] = pd.to_datetime(gas_prices['origin_date'])
        gas_prices['forecast_date'] = pd.to_datetime(gas_prices['forecast_date'])
        electricity_prices['origin_date'] = pd.to_datetime(electricity_prices['origin_date'])
        electricity_prices['forecast_date'] = pd.to_datetime(electricity_prices['forecast_date'])
        forecast_weather['origin_datetime'] = pd.to_datetime(forecast_weather['origin_datetime'])
        forecast_weather['forecast_datetime'] = pd.to_datetime(forecast_weather['forecast_datetime'])
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
        revealed_targets['datetime'] = pd.to_datetime(revealed_targets['datetime'])
        
    # Добавляем колонку заполненную следующим data_block_id
    test["data_block_id"] = cur_test_data_block_id
    revealed_targets["data_block_id"] = cur_test_data_block_id
    
    df_test            = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_new_client      = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_new_gas         = pl.from_pandas(gas_prices[gas_cols], schema_overrides=schema_gas)
    df_new_electricity = pl.from_pandas(electricity_prices[electricity_cols], schema_overrides=schema_electricity)
    df_new_forecast    = pl.from_pandas(forecast_weather[forecast_cols], schema_overrides=schema_forecast)
    df_new_historical  = pl.from_pandas(historical_weather[historical_cols], schema_overrides=schema_historical)
    df_new_target      = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    df_new_data        = pl.from_pandas(revealed_targets[df_data_cols], schema_overrides=schema_data)
    # Объединение новых данных с существующими и удаление дубликатов
    df_client          = pl.concat([df_client, df_new_client]).unique(subset=["county", "is_business", "product_type", "date"], maintain_order=True)
    df_gas             = pl.concat([df_gas, df_new_gas]).unique(subset=["forecast_date"], maintain_order=True)
    df_electricity     = pl.concat([df_electricity, df_new_electricity]).unique(subset=["forecast_date"], maintain_order=True)
    df_forecast        = pl.concat([df_forecast, df_new_forecast]).unique()
    df_historical      = pl.concat([df_historical, df_new_historical]).unique()
    df_target          = pl.concat([df_target, df_new_target]).unique()
    df_data            = pl.concat([df_data, df_new_data]).unique()
        
    if iteration__scrored:
    # if True:
        # Инициализируем итерацию обучения и предсказания
        models.init_iter(itter_n=count)
        cur_time = time.time()
        if ((cur_time < stop_train_time) or is_disable_run_time_limit):
            # Не начинаем тренировать модели заново на реальном сабмите пока в датах предсказания не появятся даты идущие в скор
            X, y = df_data.drop("target"), df_data.select("target")
            X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
            #Добавились новые строки в данные. Учим.
            df_train = to_pandas(X, y)
            models.fit(df_train=df_train)
        else:
            print('Не тренеруем модели, превышено время выполнения ноутбука:', (cur_time - notebook_starttime))
    
        # Применение функции инженерии признаков и преобразование данных обратно в pandas
        X_test = feature_eng(df_test, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
        X_test = to_pandas(X_test)
        
        # Прогнозирование с использованием модели и ограничение предсказаний нулем
        test['target'] = models.predict(X_test)
        
        # Обновление целевых значений в примере предсказания
        sample_prediction["target"] = test['target']
        
        # Отправка предсказаний в среду выполнения
        env.predict(sample_prediction)
    
        if is_local:
            # Выводим текущий скор в разных разрезах
            compare = calc_score()
            pass
    else:
        print(p_time(), 'Итерация не входит в скор, просто возвращаем sample_prediction')
        env.predict(sample_prediction)
        models.dummy_predict(sample_prediction["target"].values)
        
    count += 1
    # Переходим к следующему data_block_id на итерации тестов
    cur_test_data_block_id += 1
    print(p_time(), 'Iteration run time:', round(time.time() - iteration_start_time))
    print('')
    print('________________________________________________')
    print('')

## Анализ предсказания

### Подсчет скора

In [ ]:
if is_local:
    compare = calc_score()

In [ ]:
if is_local:
    dump(models.models['model-1'], 'models.joblib')
    pass

In [ ]:
#models = load('models.joblib')

In [ ]:
'''
count = 0
for cur_predict in models.predictions['model-1']:
    cur_predict = cur_predict * 0
    print(cur_predict)
    for model_name in models.predictions:
        #print(models.predictions[model_name][count])
        
        break
    count += 1
    
    break
'''

### График MAE по дням предсказания

In [ ]:
# выводит график средних ошибок сгруппированных по дням (точнее для блоков данных для предсказаний которые в целом эквиваленты дням)
def print_err(err_name, err_lable, err_title):
    # Группируем по data_block_id, то есть по дням и считаем отдельно для каждого дня предсказания MAE
    grouped_compare = compare.groupby('data_block_id').mean().reset_index()
    # Делаем скользящую среднюю
    grouped_compare['rolling_mean'] = grouped_compare[err_name].rolling(window=30, min_periods=1).mean()
    
    
    # Plotting the mean absolute errors
    plt.figure(figsize=(10, 8))
    #plt.bar(grouped_compare['data_block_id'], grouped_compare['abs_err'])
    plt.bar(grouped_compare['data_block_id'], grouped_compare[err_name], label=err_lable)
    plt.plot(grouped_compare['data_block_id'],
             grouped_compare['rolling_mean'],
             label='Rolling Mean (window=30)',
             color='orange',
             linestyle='-', linewidth=2)
    plt.xlabel('data_block_id')
    plt.ylabel(err_lable)
    plt.title(err_title)
    plt.legend()
    
    # Set ticks every 10 data_block_id
    tick_positions_y = np.arange(-40, max(grouped_compare[err_name]) + 1, 10)
    plt.yticks(tick_positions_y)
    plt.grid(True)
    plt.show()

In [ ]:
if is_local:
    print_err(err_name='abs_err',
              err_lable='Mean Absolute Error',
              err_title='Mean Absolute Error by data_block_id')

In [ ]:
if is_local:
    compare = compare[compare["data_block_id"] > 600]
    print_err(err_name='err',
              err_lable='Mean Error (predict - target)',
              err_title='Mean Error by data_block_id')